In [1]:
import tensorflow as tf
import numpy as np
import os
import datetime
import pandas as pd
import re
import pprint
import functools
import sys


# Create the Training dataset


In [2]:
# Import Dataframe from desktop
Filepath = '../input/dead-lyrics-5/Dead_lyrics5_CSV.csv'
df = pd.read_csv(Filepath)

#Left align text from csv
def left_justified(df):
    formatters = {}
    for li in list(df.columns):
        max = df[li].str.len().max()
        form = "{{:<{}s}}".format(max)
        formatters[li] = functools.partial(str.format, form)
    return df.to_string(formatters=formatters, index=False)

df2 = left_justified(df)


# Use regex to find and remove song titles and unwanted characters from lyrics
def remove_characters(str1):
    result = re.sub(r"[A-Z'(]{2,}[^a-z]", " ", str1)
    result2 = re.sub(r"\[*]*\\*", "", result)
    result3 = re.sub(r"((,)?\*?)", "", result2)
    result4 = re.sub(r"[0-9]", "", result3)
    return result4

no_caps = remove_characters(df2)

#Extra words/characters we dont want in lyrics data
l_string = no_caps.replace('/', ' ').replace('(note)', ' ').replace('(note )', ' ').replace('(note a)', ' ')\
.replace('(note a)', ' ').replace('(note b)', ' ').replace('(note c)', ' ').replace('(note d)', ' ')\
.replace('(note e)', ' ').replace('(note f)', ' ').replace('(note g)', ' ')

print(no_caps[:1000])



                                                                                                                                                                                   Thirty two teeth in a jawbone                                                                                                                                                                 
 Alabama cryin for none                                                                                                                                                                        
 Before I have to hit him                                                                                                                                                                      
 I hope he's got the sense to run                                                                                                                                                              
 Reason those poor girls love him                     

In [3]:
#Get rid of empty spaces in string
lyrics_split = " ".join(l_string.split())

# View the length of the data and the first 500 characters
print(len(lyrics_split))
print(lyrics_split[:500]) 

100605
Thirty two teeth in a jawbone Alabama cryin for none Before I have to hit him I hope he's got the sense to run Reason those poor girls love him Promise them anything Reason they believe him He wears a big diamond ring Alabama getaway Alabama getaway Only way to please me Turn around and leave and walk away Majordomo Billy Bojangles Sit down and have a drink with me What's this about Alabama Keeps comin back to me? Heard your plea in the courthouse Jurybox began to rock and rise Forty-nine sister


In [4]:

# Create a set of the unique characters featured in lyrics_split
#This will be our vocabulary
vocab = sorted(set(lyrics_split))

print(vocab[:100])
print(f' "\n Number of unique characters: {len(vocab)}')

[' ', '!', '"', "'", '(', ')', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '…']
 "
 Number of unique characters: 65


# Process the text

## Vectorise the text

Before training, we need to map strings to a numerical representation. To do this, lets create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in lyrics_split])

#Now you have an integer representation for each character. 
#Notice that you mapped the character as indexes from 0 to len(unique).

print('{')
for char,_ in zip(char2idx, range(65)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  "'" :   3,
  '(' :   4,
  ')' :   5,
  '-' :   6,
  '.' :   7,
  ':' :   8,
  ';' :   9,
  '?' :  10,
  'A' :  11,
  'B' :  12,
  'C' :  13,
  'D' :  14,
  'E' :  15,
  'F' :  16,
  'G' :  17,
  'H' :  18,
  'I' :  19,
  'J' :  20,
  'K' :  21,
  'L' :  22,
  'M' :  23,
  'N' :  24,
  'O' :  25,
  'P' :  26,
  'Q' :  27,
  'R' :  28,
  'S' :  29,
  'T' :  30,
  'U' :  31,
  'V' :  32,
  'W' :  33,
  'X' :  34,
  'Y' :  35,
  'Z' :  36,
  'a' :  37,
  'b' :  38,
  'c' :  39,
  'd' :  40,
  'e' :  41,
  'f' :  42,
  'g' :  43,
  'h' :  44,
  'i' :  45,
  'j' :  46,
  'k' :  47,
  'l' :  48,
  'm' :  49,
  'n' :  50,
  'o' :  51,
  'p' :  52,
  'q' :  53,
  'r' :  54,
  's' :  55,
  't' :  56,
  'u' :  57,
  'v' :  58,
  'w' :  59,
  'x' :  60,
  'y' :  61,
  'z' :  62,
  'é' :  63,
  '…' :  64,
  ...
}


In [6]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(lyrics_split[:100]), text_as_int[:100]))


"Thirty two teeth in a jawbone Alabama cryin for none Before I have to hit him I hope he's got the se" ---- characters mapped to int ---- > [30 44 45 54 56 61  0 56 59 51  0 56 41 41 56 44  0 45 50  0 37  0 46 37
 59 38 51 50 41  0 11 48 37 38 37 49 37  0 39 54 61 45 50  0 42 51 54  0
 50 51 50 41  0 12 41 42 51 54 41  0 19  0 44 37 58 41  0 56 51  0 44 45
 56  0 44 45 49  0 19  0 44 51 52 41  0 44 41  3 55  0 43 51 56  0 56 44
 41  0 55 41]


## Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [7]:
# The maximum length of sentence you want for a single input, in characters
seq_length = 100
examples_per_epoch = len(l_string)//(seq_length+1)

# Create training examples / targets
# Use the tf.data.Dataset to create a sourse dataset from input data 
# Makes it easy to perfrom transformations to the data.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])
    

T
h
i
r
t


In [8]:
# The 'batch' method lets us easily convert these individual charaters to sequences of the desired size.

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

"Thirty two teeth in a jawbone Alabama cryin for none Before I have to hit him I hope he's got the sen"
'se to run Reason those poor girls love him Promise them anything Reason they believe him He wears a b'
'ig diamond ring Alabama getaway Alabama getaway Only way to please me Turn around and leave and walk '
"away Majordomo Billy Bojangles Sit down and have a drink with me What's this about Alabama Keeps comi"
'n back to me? Heard your plea in the courthouse Jurybox began to rock and rise Forty-nine sister stat'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [9]:
# For each sequence, duplicate and shift it to form the input and target text by using the map method 
# to apply a simple function to each batch:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Thirty two teeth in a jawbone Alabama cryin for none Before I have to hit him I hope he's got the se"
Target data: "hirty two teeth in a jawbone Alabama cryin for none Before I have to hit him I hope he's got the sen"


Each index of these vectors is processed as a one time step. For the input at time step 0, the model receives the index for "T" and tries to predict the index for "h" as the next character. 

At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

Here is an example of what it is doing:


In [10]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('T')
  expected output: 44 ('h')
Step    1
  input: 44 ('h')
  expected output: 45 ('i')
Step    2
  input: 45 ('i')
  expected output: 54 ('r')
Step    3
  input: 54 ('r')
  expected output: 56 ('t')
Step    4
  input: 56 ('t')
  expected output: 61 ('y')


## Creating training batches

So we have now used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [11]:
# Batch size
BATCH_SIZE  = 10

## Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000


dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
     .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((10, 100), (10, 100)), types: (tf.int64, tf.int64)>

# Build the Model


Words seen as vectors that need to be computed in the best way possible to capture the meaning of the word itself (this is called 'embedding').

This is where Gated Recurrent Units or GRUs come in. These are cells that are able to 'remember' a certain number of previous words in a clever way. 

Finally, we use a dense layer with the logit output that gives info about the most probable word expected.

## Model Description

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

1. tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
2. tf.keras.layers.GRU: A type of RNN with size units=rnn_units. (Can also use an LSTM layer here).
3. tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [12]:
# Length of the vocabulary in characters
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)



In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape,"# (batch_size, sequence_length, vocab_size)")

(10, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (10, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (10, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (10, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get the correct character indices. This distribution is defined by the logits over the character vocabulary.

Logits are the un-normalised raw predictions (outputs) of a model. We go on to use the softmax to compute the normalised score that each instance belongs to which class.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Here is an example from the first batch:

In [15]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:



In [16]:
sampled_indices

array([31, 28, 33,  0, 23, 53,  9,  6,  9,  4, 60, 40, 38, 30, 24, 41,  1,
       24,  9, 20, 13, 10,  9, 24, 29, 29, 13, 47, 37, 55, 45, 26,  5, 13,
        8, 57, 49, 37,  4,  9,  6, 21, 27, 19, 29,  5, 63, 63, 40, 38, 33,
       36, 10, 35,  9, 12, 51, 62, 37, 32, 15, 12, 14, 58, 38,  9, 17, 31,
       49, 46, 46, 58, 23,  1, 52, 20, 49, 49,  2,  3, 55, 10, 45,  7, 34,
       25, 38, 34, 32, 11, 44, 31, 34, 14, 46, 21, 11, 16, 16, 43])

Decode these to see the text predicted by this untrained model:

In [17]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' in the pines and the sun and bright moonlight Lazing in the sunlight yes indeed Sometimes when the '

Next Char Predictions: 
 'URW Mq;-;(xdbTNe!N;JC?;NSSCkasiP)C:uma(;-KQIS)éédbWZ?Y;BozaVEBDvb;GUmjjvM!pJmm"\'s?i.XObXVAhUXDjKAFFg'


# Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input from this time step, the model predicts the class of the next character.

We also attach an optimizer, and a loss function.

The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because the model returns logits, we need to set the from_logits flag.




In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (10, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173676


Now we configure the training procedure using the tf.keras.Model.compile method. Use tf.keras.optimizers.Adam with default arguments and the cross entropy loss function.

In [19]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

Configure checkpoints:

Use a tf.keras.callbacks.ModelCheckpoint to ensure that weights are saved during training.

Then we FIT THE DATA!


In [20]:


# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints/'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
monitor ='val_accuracy')


EPOCHS = 30

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])




Epoch 1/30
99/99 [==============================] - 2s 22ms/step - loss: 2.8585 - accuracy: 0.2753
Epoch 2/30
99/99 [==============================] - 2s 22ms/step - loss: 2.1396 - accuracy: 0.3931
Epoch 3/30
99/99 [==============================] - 2s 22ms/step - loss: 1.9607 - accuracy: 0.4365
Epoch 4/30
99/99 [==============================] - 2s 22ms/step - loss: 1.8182 - accuracy: 0.4710
Epoch 5/30
99/99 [==============================] - 2s 22ms/step - loss: 1.6926 - accuracy: 0.5054
Epoch 6/30
99/99 [==============================] - 2s 22ms/step - loss: 1.5763 - accuracy: 0.5352
Epoch 7/30
99/99 [==============================] - 2s 21ms/step - loss: 1.4599 - accuracy: 0.5677
Epoch 8/30
99/99 [==============================] - 2s 23ms/step - loss: 1.3505 - accuracy: 0.5990
Epoch 9/30
99/99 [==============================] - 2s 22ms/step - loss: 1.2330 - accuracy: 0.6326
Epoch 10/30
99/99 [==============================] - 2s 22ms/step - loss: 1.1065 - accuracy: 0.6699
Epoch 11/

# Generate text
Restore the latest checkpoint.

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we would need to rebuild the model and restore the weights from the checkpoint.




In [21]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_30'

In [22]:

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## The prediction loop

The following code block generates the lyric text.

We begin by choosing a start string, which initializes the RNN state, and also set the number of characters we want to generate.

Then we get the prediction distribution of the next character using the start string and the RNN state.

Using a categorical distribution, the model calculates the index of the predicted character, which is then used as the next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one character. 

After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

In [24]:
def generate_text(model, start_string, t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 450

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        
        #Attempt to present string in lyric format with new line after each capital letter
        text_join = (start_string + ''.join(text_generated))
        result = re.sub( r"\s(?=[A-Z])", r" \n", text_join)
    return result
    

In [25]:
print(generate_text(model, start_string= "Mountain", t = 0.4))

Mountains that rise above the plain 
Wave away my confusion 
That drives this delusion 
So long 
I forgottann stars 
Don't you cry 
Dry your eyes on the wind 
La da da da da 
All 
I know is something like a bird within her sang 
All 
I know she sang a little while and then flew on 
Tell me all that you know 
I'll show you 
Snow and rain 
When the last rose of summer pricks my fingers 
And the hot sun chills me to the 
Chorus 
That's loose on the town 
Take a whole pail of 
